# Row Level Filters in Unity Catalog
## Databricks Zero to Hero - Session 36

**Objective:** Learn how to implement Row Level Security (RLS) in Databricks Unity Catalog using Row Filters. This allows you to show or hide specific rows of data in a table based on who is querying it.

**Use Case:**
Imagine a `customer` table containing data for multiple market segments (e.g., AUTOMOBILE, HOUSEHOLD, BUILDING).
*   **User A (Manager):** Should only see rows where `market_segment = 'BUILDING'`.
*   **User B (Data Engineer):** Should only see rows where `market_segment = 'HOUSEHOLD'`.
*   Instead of creating separate tables/views, we apply a **Row Filter** on the main table.

**Prerequisites:**
1.  Unity Catalog enabled workspace.
2.  Compute cluster compatible with Unity Catalog (Shared access mode recommended for RLS).
3.  Permissions to create functions and alter tables.

### 1. Setup Metadata for Permissions
Instead of hardcoding users in the filter logic (which is hard to maintain), we will create a "Mapping Table" (Metadata table). This table defines which user is allowed to see which market segment.

*   **Schema:** `dev.metadata`
*   **Table:** `allowed_mktsegment`

In [ ]:
%sql
-- Create a schema for metadata if it doesn't exist
CREATE SCHEMA IF NOT EXISTS dev.metadata;

-- Create the mapping table
CREATE TABLE IF NOT EXISTS dev.metadata.allowed_mktsegment (
    user_email STRING,
    mktsegment STRING
);

-- Clean up any old data for demo purposes
TRUNCATE TABLE dev.metadata.allowed_mktsegment;

### 2. Insert Access Rules
Let's define the rules.
*   Replace `<your_email>` with your actual Databricks login email (User 1).
*   Replace `<secondary_email>` with another user's email if you have one to test (User 2), or just define rules for yourself to see the filtering in action.

*In this example:*
*   Current User -> Allowed to see 'BUILDING'
*   Other User -> Allowed to see 'HOUSEHOLD'

In [ ]:
%sql
-- Insert data mapping.
-- logic: user_email | segment_they_can_see

INSERT INTO dev.metadata.allowed_mktsegment VALUES
(current_user(), 'BUILDING'), -- You will only see BUILDING
('data_engineer@example.com', 'HOUSEHOLD'); -- Dummy user for demo logic

-- Verify the metadata
SELECT * FROM dev.metadata.allowed_mktsegment;

### 3. Create the Row Filter Function
A Row Filter is essentially a User Defined Function (UDF) that returns a `BOOLEAN` (True/False).
*   **Input:** The column value from the table row (e.g., the market segment of that specific row).
*   **Logic:** Check if the `current_user()` exists in our metadata table associated with that input segment.
*   **Output:**
    *   `TRUE`: The user is allowed to see this row.
    *   `FALSE`: The row is hidden from the user.

In [ ]:
%sql
CREATE OR REPLACE FUNCTION dev.metadata.is_allowed_mktsegment(input_segment STRING)
RETURNS BOOLEAN
LANGUAGE SQL
RETURN EXISTS (
    SELECT 1
    FROM dev.metadata.allowed_mktsegment m
    WHERE m.mktsegment = input_segment
    AND m.user_email = current_user()
);

In [ ]:
%sql
-- Test the function manually to verify logic before applying
-- Should return TRUE because we inserted a rule for current_user -> BUILDING
SELECT dev.metadata.is_allowed_mktsegment('BUILDING') as is_access_granted;

In [ ]:
%sql
-- Should return FALSE because we did not give current_user access to AUTOMOBILE
SELECT dev.metadata.is_allowed_mktsegment('AUTOMOBILE') as is_access_granted;

### 4. Approach 1: Dynamic Views
Before modifying the physical table, a common pattern is to create a **Dynamic View**. This view applies the filter logic dynamically.

In [ ]:
%sql
-- Create a view that uses the function in the WHERE clause
CREATE OR REPLACE VIEW dev.bronze.v_customer_filtered AS
SELECT * FROM dev.bronze.customer_raw
WHERE dev.metadata.is_allowed_mktsegment(c_mktsegment);

-- Query the view
-- You should ONLY see rows where c_mktsegment = 'BUILDING'
SELECT * FROM dev.bronze.v_customer_filtered;

### 5. Approach 2: Apply Row Filter to the Table
Now, let's enforce security directly on the table. This ensures that no matter how the user queries the table (directly or via BI tools), the filter is always applied.

**Syntax:** `ALTER TABLE ... SET ROW FILTER ... ON (...)`

In [ ]:
%sql
-- Apply the function as a row filter on the specific column
ALTER TABLE dev.bronze.customer_raw
SET ROW FILTER dev.metadata.is_allowed_mktsegment ON (c_mktsegment);

### 6. Verify Results
Now, run a simple `SELECT *` on the raw table. Even though we are querying the full table, Unity Catalog intercepts the query and applies the UDF logic.

In [ ]:
%sql
-- Select from the base table
-- Result: You should only see 'BUILDING' data.
SELECT * FROM dev.bronze.customer_raw;

In [ ]:
%sql
-- Let's check distinct segments to be sure
SELECT DISTINCT c_mktsegment FROM dev.bronze.customer_raw;

### 7. How to remove the Row Filter?
If you need to remove the security policy (for example, to debug or re-architect), you can drop the filter.

In [ ]:
%sql
-- Remove the row filter
ALTER TABLE dev.bronze.customer_raw
DROP ROW FILTER;

-- Verify that full access is restored (you should see all segments now)
SELECT DISTINCT c_mktsegment FROM dev.bronze.customer_raw;

### Summary
1.  **Row Filters** allow fine-grained access control at the row level.
2.  We used a **Metadata Table** to manage permissions dynamically without changing code.
3.  We used a **SQL UDF** to encapsulate the logic checking `current_user()`.
4.  The filter is applied directly to the table using `ALTER TABLE SET ROW FILTER`.